In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [2]:
# Load the dataset from Google Drive
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'D:\Projects\CDP\Dataset\Dataset-1\Train',
    labels="inferred",
    label_mode="int",
    image_size=(28, 28),
    batch_size=32,
    color_mode="grayscale"
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'D:\Projects\CDP\Dataset\Dataset-1\Validation',
    labels="inferred",
    label_mode="int",
    image_size=(28, 28),
    batch_size=32,
    color_mode="grayscale"
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'D:\Projects\CDP\Dataset\Dataset-1\Test',
    labels="inferred",
    label_mode="int",
    image_size=(28, 28),
    batch_size=32,
    color_mode="grayscale"
)

# Normalize the pixel values to [-1, 1] for GAN training
def normalize(image, label):
    image = (image - 127.5) / 127.5
    return image, label

train_dataset = train_dataset.map(normalize)
val_dataset = val_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [3]:
# CGAN Generator Model (with label input)
def make_generator_model():
    # Label input and embedding
    label_input = layers.Input(shape=(1,))
    noise_input = layers.Input(shape=(100,))

    # Embed and reshape the label
    label_embedding = layers.Embedding(2, 50)(label_input)
    label_dense = layers.Dense(7 * 7 * 1)(label_embedding)
    label_reshape = layers.Reshape((7, 7, 1))(label_dense)

    # Noise input and reshape
    noise_dense = layers.Dense(7 * 7 * 256, use_bias=False)(noise_input)
    noise_batchnorm = layers.BatchNormalization()(noise_dense)
    noise_reshape = layers.Reshape((7, 7, 256))(noise_batchnorm)

    # Concatenate label and noise
    concatenated = layers.Concatenate()([noise_reshape, label_reshape])

    # Building the model using the functional API
    x = layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(concatenated)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    output = layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh')(x)

    return tf.keras.Model([noise_input, label_input], output)

def make_discriminator_model():
    # Label input and embedding
    label_input = layers.Input(shape=(1,))
    image_input = layers.Input(shape=(28, 28, 1))

    # Embed and reshape the label
    label_embedding = layers.Embedding(2, 50)(label_input)
    label_dense = layers.Dense(28 * 28)(label_embedding)
    label_reshape = layers.Reshape((28, 28, 1))(label_dense)

    # Concatenate label and image input
    concatenated = layers.Concatenate()([image_input, label_reshape])

    # Build the discriminator model using the functional API
    x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(concatenated)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    output = layers.Dense(1)(x)

    return tf.keras.Model([image_input, label_input], output)


# Build the generator and discriminator
generator = make_generator_model()
discriminator = make_discriminator_model()

# Loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training Loop
EPOCHS = 50
noise_dim = 100
BATCH_SIZE = 32
num_examples_to_generate = 16

In [4]:
@tf.function
def train_step(images, labels):
    noise = tf.random.normal([images.shape[0], noise_dim])  # Match batch size with images
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate images using the generator
        generated_images = generator([noise, labels], training=True)
        
        # Get real and fake output from the discriminator
        real_output = discriminator([images, labels], training=True)
        fake_output = discriminator([generated_images, labels], training=True)
        
        # Calculate generator and discriminator losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Apply gradients to the generator and discriminator
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        for image_batch, labels_batch in dataset:
            train_step(image_batch, labels_batch)

        print(f"Time for epoch {epoch + 1} is {time.time() - start} sec")
        
train(train_dataset, EPOCHS)

Time for epoch 1 is 1086.8935239315033 sec
Time for epoch 2 is 724.2298266887665 sec
Time for epoch 3 is 660.2270333766937 sec
Time for epoch 4 is 584.8917462825775 sec
Time for epoch 5 is 767.5784523487091 sec
Time for epoch 6 is 604.1639895439148 sec
Time for epoch 7 is 595.9945447444916 sec
Time for epoch 8 is 592.8673405647278 sec
Time for epoch 9 is 591.1071863174438 sec
Time for epoch 10 is 592.9089303016663 sec
Time for epoch 11 is 598.4381096363068 sec
Time for epoch 12 is 597.5404062271118 sec
Time for epoch 13 is 591.3100609779358 sec
Time for epoch 14 is 594.1696751117706 sec
Time for epoch 15 is 589.2087919712067 sec
Time for epoch 16 is 589.8747265338898 sec
Time for epoch 17 is 589.4686923027039 sec
Time for epoch 18 is 592.6251075267792 sec
Time for epoch 19 is 587.7482159137726 sec
Time for epoch 20 is 587.30588722229 sec
Time for epoch 21 is 591.4061667919159 sec
Time for epoch 22 is 623.9222757816315 sec
Time for epoch 23 is 619.6030957698822 sec
Time for epoch 24 is 

In [5]:
# Accuracy Metrics


# Evaluation Metrics
y_true = []
y_pred = []
for test_images, test_labels in test_dataset:
    real_output = discriminator([test_images, test_labels], training=False)
    fake_images = generator([tf.random.normal([len(test_images), noise_dim]), test_labels], training=False)
    fake_output = discriminator([fake_images, test_labels], training=False)

    y_true.extend([1] * len(test_images) + [0] * len(fake_images))
    y_pred.extend([1 if p > 0 else 0 for p in real_output] + [1 if p > 0 else 0 for p in fake_output])

# Confusion Matrix and Metrics
conf_matrix = confusion_matrix(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Confusion Matrix:
[[10353   552]
 [ 5171  5734]]
Accuracy: 0.7375974323704723
Precision: 0.9121858097359211
Recall: 0.5258138468592389
F1-Score: 0.6670932464661742


In [6]:
# Save the trained model
generator.save('D:\Projects\CDP\Model\gan_generator_model.h5')
discriminator.save('D:\Projects\CDP\Model\gan_discriminator_model.h5')

In [13]:
# Predict an image

import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained generator and discriminator models
generator = tf.keras.models.load_model('D:\Projects\CDP\Model\gan_generator_model.h5')
discriminator = tf.keras.models.load_model('D:\Projects\CDP\Model\gan_discriminator_model.h5')

# Normalize the image to [-1, 1] as we did during training
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
    img = image.img_to_array(img)
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict if an image is real or fake
def predict_real_or_fake(img_path, label):
    img = preprocess_image(img_path)

    # Add the label as a batch of size 1 (e.g., label 0 or 1)
    label = np.array([label]).reshape(-1, 1)
    
    # Pass the image and label to the discriminator
    prediction = discriminator([img, label], training=False)

    # If prediction > 0.5, it predicts "real"; otherwise, "fake"
    if prediction > 0:
        print("The image is predicted as REAL.")
    else:
        print("The image is predicted as FAKE.")

# Example usage
sample_image_path = 'D:/Projects/CDP/fake-image.webp'  # Replace with the path to your sample image
sample_label = 1  # Label associated with the real image (change accordingly)
predict_real_or_fake(sample_image_path, sample_label)


The image is predicted as REAL.
